<a href="https://colab.research.google.com/github/fortune-max/M4-python-refresher-ml/blob/main/Copy_of_Day_11_automl_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
tfrom google.colab import drive

drive.mount('/content/gdrive')

!cp gdrive/MyDrive/DS404/Day_11_materials/* .
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
gdrive	imports.py  sample_data  titanic.csv


In [ ]:
%run imports.py

## Parameters search

In [ ]:
titanic_df = pd.read_csv('titanic.csv')

In [ ]:
from sklearn import model_selection

In [ ]:
X_train,X_test, y_train, y_test = model_selection.train_test_split(
    titanic_df.drop('Survived', axis=1),
    titanic_df['Survived'])

In [ ]:
X_train

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
539,540,1,"Frolicher, Miss. Hedwig Margaritha",female,22.0,0,2,13568,49.5000,B39,C
159,160,3,"Sage, Master. Thomas Henry",male,NaN,8,2,CA. 2343,69.5500,NaN,S
216,217,3,"Honkanen, Miss. Eliina",female,27.0,0,0,STON/O2. 3101283,7.9250,NaN,S
751,752,3,"Moor, Master. Meier",male,6.0,0,1,392096,12.4750,E121,S
514,515,3,"Coleff, Mr. Satio",male,24.0,0,0,349209,7.4958,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
191,192,2,"Carbines, Mr. William",male,19.0,0,0,28424,13.0000,NaN,S
342,343,2,"Collander, Mr. Erik Gustaf",male,28.0,0,0,248740,13.0000,NaN,S
680,681,3,"Peters, Miss. Katie",female,NaN,0,0,330935,8.1375,NaN,Q
139,140,1,"Giglio, Mr. Victor",male,24.0,0,0,PC 17593,79.2000,B86,C


#### for classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier()

In [ ]:
grid_param = {
    'n_estimators': [100, 300, 500, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}

In [ ]:
from sklearn import model_selection

### Grid Search

In [ ]:
grid_search_classifier = model_selection.GridSearchCV(estimator=classifier,
                     param_grid=grid_param,
                     scoring='accuracy',
                     cv=5,
                     n_jobs=-1)
grid_search_classifier

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'n_estimators': [100, 300, 500, 800, 1000]},
             scoring='accuracy')

In [ ]:
%%time
grid_search_classifier.fit(X_train[ ['Pclass', 'SibSp','Fare' ] ], y_train)

CPU times: user 2.4 s, sys: 110 ms, total: 2.51 s
Wall time: 1min 16s


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'n_estimators': [100, 300, 500, 800, 1000]},
             scoring='accuracy')

In [ ]:
best_parameters = grid_search_classifier.best_params_
print(best_parameters)

{'bootstrap': True, 'criterion': 'entropy', 'n_estimators': 1000}


In [ ]:
grid_search_classifier.best_score_

0.6705981371338795

In [ ]:
grid_search_classifier.cv_results_

{'mean_fit_time': array([0.26321416, 0.78713927, 1.2664114 , 1.96815248, 2.46601233,
        0.26504087, 1.18209071, 1.89387321, 2.05865355, 3.42043643,
        0.20049038, 0.57493072, 0.95504727, 1.5319478 , 1.91964293,
        0.22141719, 0.65709167, 1.10312214, 1.77510481, 2.14945941]),
 'std_fit_time': array([0.02001154, 0.02077161, 0.02451371, 0.04056276, 0.03226828,
        0.00740773, 0.31994422, 0.70452959, 0.03229831, 0.65062258,
        0.0036143 , 0.01165168, 0.0235769 , 0.01589568, 0.03234015,
        0.00876814, 0.0168834 , 0.0218335 , 0.02694068, 0.11998224]),
 'mean_score_time': array([0.02310591, 0.06267686, 0.0996161 , 0.16251731, 0.20054116,
        0.02269745, 0.11571059, 0.11799417, 0.15668402, 0.25541601,
        0.02534814, 0.0591197 , 0.10129261, 0.15975394, 0.20634432,
        0.0220602 , 0.06088176, 0.09662004, 0.16072993, 0.17961211]),
 'std_score_time': array([0.00047413, 0.00406834, 0.00303292, 0.00675065, 0.00788826,
        0.00181509, 0.02833489, 0.022561

### Random search

In [ ]:
random_search_classifier = model_selection.RandomizedSearchCV(estimator=classifier,
                     param_distributions=grid_param,
                     scoring='accuracy',
                     cv=5,
                     n_jobs=-1)

In [ ]:
%%time
random_search_classifier.fit(X_train[ ['Pclass', 'SibSp','Fare' ] ], y_train)

CPU times: user 1.75 s, sys: 118 ms, total: 1.87 s
Wall time: 39.1 s


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'n_estimators': [100, 300, 500, 800,
                                                         1000]},
                   scoring='accuracy')

In [ ]:
best_parameters = random_search_classifier.best_params_
print(best_parameters)

{'n_estimators': 800, 'criterion': 'entropy', 'bootstrap': True}


In [ ]:
random_search_classifier.best_score_

0.6721131186174392

In [ ]:
grid_search_classifier.cv_results_

{'mean_fit_time': array([0.26321416, 0.78713927, 1.2664114 , 1.96815248, 2.46601233,
        0.26504087, 1.18209071, 1.89387321, 2.05865355, 3.42043643,
        0.20049038, 0.57493072, 0.95504727, 1.5319478 , 1.91964293,
        0.22141719, 0.65709167, 1.10312214, 1.77510481, 2.14945941]),
 'std_fit_time': array([0.02001154, 0.02077161, 0.02451371, 0.04056276, 0.03226828,
        0.00740773, 0.31994422, 0.70452959, 0.03229831, 0.65062258,
        0.0036143 , 0.01165168, 0.0235769 , 0.01589568, 0.03234015,
        0.00876814, 0.0168834 , 0.0218335 , 0.02694068, 0.11998224]),
 'mean_score_time': array([0.02310591, 0.06267686, 0.0996161 , 0.16251731, 0.20054116,
        0.02269745, 0.11571059, 0.11799417, 0.15668402, 0.25541601,
        0.02534814, 0.0591197 , 0.10129261, 0.15975394, 0.20634432,
        0.0220602 , 0.06088176, 0.09662004, 0.16072993, 0.17961211]),
 'std_score_time': array([0.00047413, 0.00406834, 0.00303292, 0.00675065, 0.00788826,
        0.00181509, 0.02833489, 0.022561

#### for pipeline

# AutoML

### auto-sklearn

In [ ]:
!pip install auto-sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.5 MB 4.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.9 MB 45.8 MB/s 
     |████████████████████████████████| 24.9 MB 1.5 MB/s 
     |████████████████████████████████| 260 kB 59.0 MB/s 
     |████████████████████████████████| 4.4 MB 46.9 MB/s 
     |████████████████████████████████| 46 kB 4.5 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.15.0-py3-none-any.whl size=6641945 sha256=0022bfb00aeeabcfa8993f934341bc76239c67b176eb52f800bd1715ad9b55f8
  Stored in directory: /root/.cache/pip/wheels/c5/11/0e/aeac2cee929fa4388e528737b49deb2eea05486e6f80d61c1a
  Created wheel for pynisher: filename=pynisher-0.6.4-py3-none-any.whl size=7044 sha256=9d885e7d00901121736bcb8b383cce04f6e03c11edaf4d31cedb8a2e954c54ac
  Sto

In [ ]:
import autosklearn.classification

In [ ]:
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=60)

In [ ]:
%%time
automl.fit(X_train[ ['Pclass', 'SibSp','Fare' ] ], y_train)

CPU times: user 3.41 s, sys: 627 ms, total: 4.03 s
Wall time: 57.1 s


AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      per_run_time_limit=6, time_left_for_this_task=60)

In [ ]:
from sklearn import metrics

In [ ]:
y_hat = automl.predict(X_test[ ['Pclass', 'SibSp','Fare' ] ])
print("Accuracy score", metrics.accuracy_score(y_test, y_hat))

Accuracy score 0.6995515695067265


In [ ]:
automl.show_models()

{2: {'model_id': 2,
  'rank': 1,
  'cost': 0.32579185520361986,
  'ensemble_weight': 0.06,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x7fe0455dff10>,
  'balancing': Balancing(random_state=1),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x7fe0455df7c0>,
  'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice at 0x7fe045a09a60>,
  'sklearn_classifier': RandomForestClassifier(max_features=1, n_estimators=512, n_jobs=1,
                         random_state=1, warm_start=True)},
 5: {'model_id': 5,
  'rank': 2,
  'cost': 0.3529411764705882,
  'ensemble_weight': 0.02,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x7fe0459e2f70>,
  'balancing': Balancing(random_state=1, strategy='weighting'),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocess

### TPOT

In [ ]:
!pip install tpot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 3.4 MB/s 
     |████████████████████████████████| 139 kB 38.2 MB/s 
     |████████████████████████████████| 193.6 MB 52 kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11953 sha256=11ad61f3a839982e59e9d0a24d9b052e32927e3641f57739c9a84f3225cd529c
  Stored in directory: /root/.cache/pip/wheels/a8/bb/8f/6b9328d23c2dcedbfeb8498b9f650d55d463089e3b8fc0bfb2
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
from tpot import TPOTClassifier

In [ ]:
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2)
tpot.fit(X_train[ ['Pclass', 'SibSp','Fare' ] ], y_train)
print(tpot.score(X_test[ ['Pclass', 'SibSp','Fare' ] ], y_test))

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7035910672202895

Generation 2 - Current best internal CV score: 0.7035910672202895

Generation 3 - Current best internal CV score: 0.7065424755919649

Generation 4 - Current best internal CV score: 0.7065424755919649

Generation 5 - Current best internal CV score: 0.7065424755919649

Best pipeline: XGBClassifier(input_matrix, learning_rate=0.01, max_depth=1, min_child_weight=11, n_estimators=100, n_jobs=1, subsample=0.25, verbosity=0)
0.6860986547085202


In [ ]:
tpot.export('my_tpot_code.py')

In [ ]:
%cat my_tpot_code.py


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=None)

# Average CV score on the training set was: 0.7065424755919649
exported_pipeline = XGBClassifier(learning_rate=0.01, max_depth=1, min_child_weight=11, n_estimators=100, n_jobs=1, subsample=0.25, verbosity=0)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


## Lazypredict

In [ ]:
!pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from lazypredict.Supervised import LazyClassifier

In [ ]:
X_train,X_test, y_train, y_test = model_selection.train_test_split(
    titanic_df.drop(['Survived', 'Name', 'Ticket','Cabin','Embarked'], axis=1),
    titanic_df['Survived'])

In [ ]:
lazy_classifier = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)

models,predictions = lazy_classifier.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████| 29/29 [00:03<00:00,  7.47it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
SVC                                0.87               0.84     0.84      0.86   
NuSVC                              0.87               0.84     0.84      0.86   
RandomForestClassifier             0.86               0.83     0.83      0.85   
XGBClassifier                      0.83               0.81     0.81      0.83   
ExtraTreesClassifier               0.83               0.81     0.81      0.83   
KNeighborsClassifier               0.83               0.81     0.81      0.83   
LGBMClassifier                     0.83               0.80     0.80      0.82   
BaggingClassifier                  0.83               0.80     0.80      0.83   
LabelSpreading                     0.81               0.80     0.80      0.81   
LinearDiscriminantAnalysis         0.83               0.80     0.80      0.83   
RidgeClassifierCV           